In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.6 MB/s eta 0:00:00


# Koala 7B loading in 8bit on a T4

In [2]:
!nvidia-smi

Sun Jul 30 12:35:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
!nvidia-smi

Sun Jul 30 12:41:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |   7999MiB / 15360MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import textwrap

pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [8]:
%%time
output = pipe('BEGINNING OF CONVERSATION: USER: \
what do you think of new large langauge models trend? please, consider GPUS supply shortage.')
print(wrap_text_preserve_newlines(output[0]['generated_text']))


BEGINNING OF CONVERSATION: USER: what do you think of new large langauge models trend? please, consider GPUS
supply shortage.
As a language model AI, I don't have personal opinions or preferences. However, the development and use of
large language models has been an ongoing trend in recent years due to their potential applications in various
fields such as natural language processing, machine translation, and text generation. The increasing
availability of GPUs has also made it possible for these models to be trained more efficiently.

It is true that there may be concerns about the impact of large language models on the global economy,
including the potential for job displacement. However, it is important to note that these models are not
designed to replace human workers but rather to augment them by automating certain tasks and providing
additional insights into complex data sets. Additionally, the development of large language models requires
significant expertise and resources, wh

In [9]:
%%time
output = pipe('BEGINNING OF CONVERSATION: USER: \
how to make lemon juice.')
print(wrap_text_preserve_newlines(output[0]['generated_text']))


BEGINNING OF CONVERSATION: USER: how to make lemon juice.
To make lemon juice, you will need a fresh lemon and a blender or food processor. Cut the lemon into small
pieces and remove any seeds or peel. Blend or process the lemon until it is smooth. Strain the mixture through
a fine mesh sieve to catch any pulp or seeds that may have been left behind. The resulting liquid should be
clear and tart, with a refreshing citrus flavor. You can adjust the amount of sugar or other ingredients as
needed to taste.
CPU times: user 26 s, sys: 25.6 ms, total: 26 s
Wall time: 28.9 s


In [ ]:
%%time
output = pipe('BEGINNING OF CONVERSATION: USER: \
كيف تصنع عصير ليمون؟.')
print(wrap_text_preserve_newlines(output[0]['generated_text']))

KeyboardInterrupt: ignored

In [ ]:
%%time
output = pipe('BEGINNING OF CONVERSATION: USER: What is the capital of England? \n')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: What is the capital of England?
Answer: The capital city of England is London.
CPU times: user 2.04 s, sys: 10.7 ms, total: 2.05 s
Wall time: 2.05 s
